In [ ]:
#unzip all files

import zipfile
import os

dir_path = "C:/Users/ewiggans/OneDrive - Lincoln Institute of Land Policy/Desktop/GeoConnexMap/data/2000Census"

# Loop through each file in the directory
for file_name in os.listdir(dir_path):
    # Check if the file is a zip file
    if file_name.endswith('.zip'):
        # Open the zip file
        with zipfile.ZipFile(os.path.join(dir_path, file_name), 'r') as zip_ref:
            # Extract all the files in the zip file to the same directory
            zip_ref.extractall(dir_path)
print("Unzipped files")

In [ ]:
# Import necessary libraries
import os
import requests
from bs4 
import BeautifulSoup

# List of state abbreviations in lowercase
states = ['al', 'ak', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'dc', 'fl',
          'ga', 'hi', 'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me',
          'md', 'ma', 'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh',
          'nj', 'nm', 'ny', 'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'ri',
          'sc', 'sd', 'tn', 'tx', 'ut', 'vt', 'va', 'wa', 'wv', 'wi', 'wy']

# Parent directory URL
parent_url = 'https://www2.census.gov/geo/tiger/tiger2k/'

# Loop through each state abbreviation
for state in states:
    # Construct URL for the state
    state_url = parent_url + state.lower() + '/'
    print(f"Downloading zip files for state: {state_url}")
    
    # Get the HTML content of the state directory page
    response = requests.get(state_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all links on the page
    links = soup.find_all('a')
    
    # Get a list of all zip file links in the state directory
    zip_links = [link.get('href') for link in links if link.get('href') and link.get('href').endswith('.zip')]
    
    # Loop through each zip file link and download the file
    for zip_link in zip_links:
        # Extract the county code from the file name
        county_code = zip_link.split('.')[0][-5:]
        # Construct URL for the zip file
        zip_url = state_url + zip_link
        # Construct the file name for the downloaded zip file
        zip_file_name = f"tgr{county_code}.zip"
        # Download the zip file to the specified location
        response = requests.get(zip_url)
        with open(os.path.join('C:/Users/ewiggans/OneDrive - Lincoln Institute of Land Policy/Desktop/GeoConnexMap/data/2000Census', zip_file_name), 'wb') as f:
            f.write(response.content)
    
    print(f"Zip files downloaded for state: {state}\n")

print("Download complete")


In [ ]:
##Script to intersect 2000 Census Blocks with NHDPlusV2 catchments
##2/14/2022

#Data Sources - 
##Using: https://www2.census.gov/geo/tiger/tiger2k/ as 2000 Census Blocks
##Catchments - Using: https://www.sciencebase.gov/catalog/item/61295190d34e40dd9c06bcd7


##Intersect Census Blocks for 2000 with NHDPlus V2 Catchments

##Need fields Census block year/id, 
#the catchment COMID, the % area of the census block that the intersection is, 
#and the % area of the catchment that the intersection is
import arcpy
import datetime
from arcpy import env
import time

# Enable overwrite output
arcpy.env.overwriteOutput = True

# Set the workspace to the folder where the census block data is held
env.workspace = r'C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census'

# List all the shapefiles in the folder
shapefiles = arcpy.ListFeatureClasses(feature_type='Polygon')

# Print start time
start_time = datetime.datetime.now()
print(f'Start time: {start_time}')

# Add a field called "Area_sqkm" to each shapefile and calculate the area in square kilometers
for shapefile in shapefiles:
    arcpy.AddField_management(shapefile, "Area_sqkm", "DOUBLE")
    arcpy.CalculateGeometryAttributes_management(shapefile, [["Area_sqkm", "AREA_GEODESIC"]], "", "SQUARE_KILOMETERS")
    print(f'{shapefile} processed.')

# add delay of 5 seconds
time.sleep(5)

# Intersect each shapefile with the national NHD catchments dataset
catchment_dataset = r'C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\GeoConnex Map\GeoConnex Map.gdb\main_reference_catchments'
for shapefile in shapefiles:
    output_feature_class = f"inter_{shapefile}"
    arcpy.Intersect_analysis([shapefile, catchment_dataset], output_feature_class, "ALL", "", "INPUT")
    print(f'{output_feature_class} created.')

# add delay of 5 seconds
time.sleep(5)
    
# Print total time elapsed
end_time = datetime.datetime.now()
total_time = end_time - start_time
print(f'Total time elapsed: {total_time}')


In [ ]:
#This works but got stuck on Cali and Oregon - solution was repair geometry, and pairwise intersect to resolve

# Import arcpy module
import arcpy
import os
import time

# Set the workspace
arcpy.env.workspace = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\Rscript"

# Set the output workspace
output_dir = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs"
output_gdb_name = "2000Census.gdb"
output_gdb = os.path.join(output_dir, output_gdb_name)

# Create the output geodatabase
if not arcpy.Exists(output_gdb):
    arcpy.CreateFileGDB_management(output_dir, output_gdb_name)

# Get a list of all the geopackages in the workspace
geopackages = arcpy.ListFiles("*.gpkg")

# Loop through each geopackage
for geopackage in geopackages:

    # Get the name of the geopackage without the .gpkg extension
    geopackage_name = os.path.splitext(geopackage)[0]

    # Create a feature class name from the geopackage name
    state_abbr = geopackage_name.split("_")[0]
    fc_name = f"{state_abbr}_2000CensusBlocks"

    # Convert the geopackage to a feature class
    fc = os.path.join(geopackage, f"main.{fc_name}")
    arcpy.FeatureClassToFeatureClass_conversion(fc, output_gdb, fc_name)

    # Add a new field called "Area_sqkm" and calculate the geometry in square kilometers
    arcpy.AddField_management(fc_name, "Area_sqkm", "DOUBLE")
    arcpy.CalculateGeometryAttributes_management(fc_name, [["Area_sqkm", "AREA_GEODESIC"]], "", "SQUARE_KILOMETERS")

    # Intersect the feature class with a catchment dataset
    catchment_fc = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\GeoConnex Map\GeoConnex Map.gdb\main_reference_catchments"
    output_intersect_fc = f"{fc_name}_catchment_intersect"
    arcpy.Intersect_analysis([fc_name, catchment_fc], output_intersect_fc)

# Print a message when the script has finished running
print("Script completed successfully.")


In [ ]:
import os
import arcpy
import time
import winsound

# Start time
start_time = time.time()

# Set input/output paths
input_folder = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\Rscript"
output_folder = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs"
output_gdb = os.path.join(output_folder, "2000Census.gdb")
catchment_dataset = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\GeoConnex Map\GeoConnex Map.gdb\main_reference_catchments"

# Create output geodatabase if it does not exist
if not arcpy.Exists(output_gdb):
    arcpy.CreateFileGDB_management(output_folder, "2000Census.gdb")

# Loop through geopackages in input folder
for gpkg in os.listdir(input_folder):
    if gpkg.endswith(".gpkg"):
        gpkg_path = os.path.join(input_folder, gpkg)
        state = gpkg.split("_")[0]
        fc_name = f"{state}_2000CensusBlocks"
        output_fc = os.path.join(output_gdb, fc_name)
        
        # Convert geopackage to feature class
        try:
            arcpy.FeatureClassToFeatureClass_conversion(gpkg_path + "/main." + fc_name, output_gdb, fc_name)
            print(f"Converted {gpkg} to feature class successfully")
        except Exception as e:
            print(f"Error converting {gpkg} to feature class: {e}")
            continue
        
        # Add field for square kilometer calculation
        try:
            arcpy.AddField_management(output_fc, "Area_sqkm", "DOUBLE")
            print(f"Added field to {fc_name} successfully")
        except Exception as e:
            print(f"Error adding field to {fc_name}: {e}")
            continue
        
        # Calculate geometry in square kilometers
        try:
            arcpy.CalculateGeometryAttributes_management(output_fc, [["Area_sqkm", "AREA_GEODESIC"]], "KILOMETERS")
            print(f"Calculated geometry in square kilometers for {fc_name} successfully")
        except Exception as e:
            print(f"Error calculating geometry in square kilometers for {fc_name}: {e}")
            continue
        
        # Intersect with catchment dataset
        try:
            intersect_output = os.path.join(output_gdb, f"{fc_name}_intersect")
            arcpy.Intersect_analysis([output_fc, catchment_dataset], intersect_output)
            print(f"Intersected {fc_name} with catchment dataset successfully")
        except Exception as e:
            print(f"Error intersecting {fc_name} with catchment dataset: {e}")
            continue

# Finish time
finish_time = time.time()

# Total time elapsed
total_time = finish_time - start_time
print(f"Total time elapsed: {total_time} seconds")

In [ ]:
#Calculate area of intersection
#Area_sqkm is the block area
#AREASQKM is the catchment area
#inter_sqkm is going to be area of the intersection

import arcpy
import os
import time

# Set the workspace to the directory containing the shapefiles
workspace = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb"
arcpy.env.workspace = workspace

# Start the timer
start_time = time.time()

# Get a list of all shapefiles in the workspace that start with "inter"
shapefiles = [os.path.join(workspace, file) for file in os.listdir(workspace) if file.endswith("_intersect")]

# Loop through each shapefile
for shapefile in shapefiles:
    print("Processing " + shapefile)
    
    # Add the "inter_sqkm" field
    arcpy.AddField_management(shapefile, "inter_sqkm", "DOUBLE")
    
    # Calculate the area of each feature in square kilometers and store it in the "inter_sqkm" field
    arcpy.CalculateGeometryAttributes_management(shapefile, [["inter_sqkm", "AREA_GEODESIC"]], area_unit="SQUARE_KILOMETERS")
    
    # Add the fields for calculating percent intersections
    arcpy.AddField_management(shapefile, "P_c_in_bk", "DOUBLE")
    arcpy.AddField_management(shapefile, "P_bk_in_c", "DOUBLE")
    
    # Loop through each shapefile
for shapefile in shapefiles:
    print("Processing " + shapefile)
    
    # Calculate the "P_c_in_bk" field ie the percent of the catchment in a block
    arcpy.AddField_management(shapefile, "P_c_in_bk", "DOUBLE")
    arcpy.CalculateField_management(shapefile, "P_c_in_bk", "(!inter_sqkm! / !AREASQKM!) * 100", "PYTHON3")
    
    # Calculate the "P_bk_in_c" field ie the percent of a block in a catchment 
    arcpy.AddField_management(shapefile, "P_bk_in_c", "DOUBLE")
    arcpy.CalculateField_management(shapefile, "P_bk_in_c", "(!inter_sqkm! / !Area_sqkm!) * 100", "PYTHON3")
# Stop the timer
    
print("Area calculations complete")

print("Now merging into one file...")
# Use same list of files as above

# Merge all shapefiles that start with "inter"
merged_file = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\USA_2000_merge"
arcpy.Merge_management(shapefiles, merged_file)
# Stop the timer
stop_time = time.time()
# Calculate the elapsed time
elapsed_time = stop_time - start_time
# Print a message with the elapsed time
print("completed successfully in " + str(round(elapsed_time, 2)) + " seconds.")


In [ ]:
#Calculate area of intersection
#Area_sqkm is the block area
#AREASQKM is the catchment area
#inter_sqkm is going to be area of the intersection

import arcpy
import os
import time

# Set the workspace to the directory containing the feature classes
workspace = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb"
arcpy.env.workspace = workspace

# Enable overwrite output
arcpy.env.overwriteOutput = True


# Start the timer
start_time = time.time()

# Get a list of all feature classes in the workspace that start with "inter"
feature_classes = [os.path.join(workspace, fc) for fc in arcpy.ListFeatureClasses("*_intersect")]

# Loop through each feature class
for fc in feature_classes:
    print("Processing " + fc)
    
    #repair geometry as a precaution
    arcpy.management.RepairGeometry(
    in_features=fc,
    delete_null="DELETE_NULL",
    validation_method="ESRI")
        
    # Add the "inter_sqkm" field
    arcpy.AddField_management(fc, "inter_sqkm", "DOUBLE")
    
    # Calculate the area of each feature in square kilometers and store it in the "inter_sqkm" field
    arcpy.CalculateGeometryAttributes_management(fc, [["inter_sqkm", "AREA_GEODESIC"]], area_unit="SQUARE_KILOMETERS")
    
    # Add the fields for calculating percent intersections
    arcpy.AddField_management(fc, "P_c_in_bk", "DOUBLE")
    arcpy.AddField_management(fc, "P_bk_in_c", "DOUBLE")
    
    # Loop through each feature class
for fc in feature_classes:
    print("Processing " + fc)
    
    # Calculate the "P_c_in_bk" field ie the percent of the catchment in a block
    arcpy.AddField_management(fc, "P_c_in_bk", "DOUBLE")
    arcpy.CalculateField_management(fc, "P_c_in_bk", "(!inter_sqkm! / !AREASQKM!) * 100", "PYTHON3")
    
    # Calculate the "P_bk_in_c" field ie the percent of a block in a catchment 
    arcpy.AddField_management(fc, "P_bk_in_c", "DOUBLE")
    arcpy.CalculateField_management(fc, "P_bk_in_c", "(!inter_sqkm! / !Area_sqkm!) * 100", "PYTHON3")
    
# Stop the timer
print("Area calculations complete")

print("Now merging into one feature class...")
Use same list of feature classes as above


# Merge all feature classes, the states that start with "inter"
merged_fc = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\USA_2000_merge_mod"

arcpy.management.Merge(
    inputs=feature_classes
    output=merged_fc,
    field_mappings=r'STATEFP00 "STATEFP00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,STATEFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,STATEFP00,0,2147483647;COUNTYFP00 "COUNTYFP00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,COUNTYFP00,0,2147483647;TRACTCE00 "TRACTCE00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,TRACTCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,TRACTCE00,0,2147483647;BLOCKCE00 "BLOCKCE00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,BLOCKCE00,0,2147483647;NAME00 "NAME00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,NAME00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,NAME00,0,2147483647;MTFCC00 "MTFCC00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,MTFCC00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,MTFCC00,0,2147483647;UR00 "UR00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,UR00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,UR00,0,2147483647;UACE00 "UACE00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,UACE00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,UACE00,0,2147483647;FUNCSTAT00 "FUNCSTAT00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,FUNCSTAT00,0,2147483647;ALAND00 "ALAND00" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,ALAND00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,ALAND00,-1,-1;AWATER00 "AWATER00" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,AWATER00,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,AWATER00,-1,-1;INTPTLAT00 "INTPTLAT00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,INTPTLAT00,0,2147483647;INTPTLON00 "INTPTLON00" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,INTPTLON00,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,INTPTLON00,0,2147483647;COUNTYFP "COUNTYFP" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,COUNTYFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,COUNTYFP,0,2147483647;STATEFP "STATEFP" true true false 2147483647 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,STATEFP,0,2147483647,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,STATEFP,0,2147483647;Area_sqkm "Area_sqkm" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,Area_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,Area_sqkm,-1,-1;FID_main_reference_catchments "FID_main_reference_catchments" true true false 4 Long 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,FID_main_reference_catchments,-1,-1;fid_1 "fid_1" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,fid_1,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,fid_1,-1,-1;FEATUREID "FEATUREID" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,FEATUREID,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,FEATUREID,-1,-1;AREASQKM "AREASQKM" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,AREASQKM,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,AREASQKM,-1,-1;VPUID "VPUID" true true false 254 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,VPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,VPUID,0,254;RPUID "RPUID" true true false 254 Text 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,RPUID,0,254,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,RPUID,0,254;hy_cats "hy_cats" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,hy_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,hy_cats,-1,-1;full_cats "full_cats" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,full_cats,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,full_cats,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,Shape_Length,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,Shape_Area,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,Shape_Area,-1,-1;inter_sqkm "inter_sqkm" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,inter_sqkm,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,inter_sqkm,-1,-1;P_c_in_bk "P_c_in_bk" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,P_c_in_bk,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,P_c_in_bk,-1,-1;P_bk_in_c "P_bk_in_c" true true false 8 Double 0 0,First,#,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AK_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AL_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AR_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\AZ_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CO_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CT_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\DE_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\FL_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\GA_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\HI_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IA_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ID_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IL_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\IN_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KS_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\KY_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\LA_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MA_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MD_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ME_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MI_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MN_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MO_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MS_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\MT_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NC_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\ND_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NE_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NH_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NJ_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NM_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NV_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\NY_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OH_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OK_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\PA_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\RI_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SC_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\SD_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TN_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\TX_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\UT_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VA_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\VT_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WA_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WI_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WV_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\WY_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\CA_2000CensusBlocks_intersect,P_bk_in_c,-1,-1,C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\OR_2000CensusBlocks_intersect,P_bk_in_c,-1,-1',
    add_source="NO_SOURCE_INFO"
)

# Stop the timer
stop_time = time.time()
# Calculate the elapsed time
elapsed_time = stop_time - start_time
# Print a message with the elapsed time
print("completed successfully in " + str(round(elapsed_time, 2)) + " seconds.")

In [ ]:
##Updated final code for updating uri/url, https://geoconnex.us/iow/nhdpv2-census/2000/comid 
###This will add the required fields, calculate them, rename as necessary, and remove unnecessary ones.
# # final steps to produce data
import arcpy
import winsound
import time
# Set the input and output feature classes

input_fc = r"C:\Users\ewiggans\OneDrive - Lincoln Institute of Land Policy\Desktop\GeoConnexMap\data\2000Census\PythonOutputs\2000Census.gdb\USA_2000_merge_fields"
output_fc = r"C:\Users\ewiggans\GeoConnexC\GeoConnex C Drive\GeoConnex C Drive.gdb\USA_2000_merge_centroids"

# Start the timer
start_time = time.time()
print("started at")
print(start_time)


# Add new fields
arcpy.AddField_management(input_fc, "FEATUREID_calc", "TEXT")
arcpy.AddField_management(input_fc, "FEATUREID_int", "LONG")
arcpy.AddField_management(input_fc, "BLKIDFP00_calc", "TEXT")

# Calculate new fields
arcpy.CalculateField_management(input_fc, "FEATUREID_int", "int(!FEATUREID!)", "PYTHON3")
arcpy.CalculateField_management(input_fc, "FEATUREID_calc", "str(!FEATUREID!)", "PYTHON3")
arcpy.CalculateField_management(input_fc, "FEATUREID_calc", "(!FEATUREID_calc!).rstrip('.0')", "PYTHON3")
arcpy.CalculateField_management(input_fc, "BLKIDFP00_calc", "str(!BLKIDFP00!)", "PYTHON3")

# Add the "uri", "id", and "name" fields
arcpy.AddField_management(input_fc, "uri", "TEXT")
arcpy.AddField_management(input_fc, "id", "TEXT")
arcpy.AddField_management(input_fc, "name", "TEXT")
print("Fields added")

# Calculate the "uri" field
feature_id = "(!FEATUREID_calc!)"
geoid20 = "(!BLKIDFP00_calc!)"
base_url = '"https://geoconnex.us/iow/nhgf-census/2000/comid/"'
block_url = "/block/"

#thank you Ben, four for you Ben
codeblock = """
def url_join(*parts: list) -> str:
    return '/'.join([str(p).strip().strip('/') for p in parts])
"""

expression = """url_join("https://geoconnex.us/iow/nhdpv2-census/2000/comid", !FEATUREID_calc!, "block", !BLKIDFP00_calc!)"""
arcpy.management.CalculateField(input_fc, "uri", expression, "PYTHON3", codeblock)
print("uri calculated")

# Calculate the "id" field
arcpy.management.CalculateField(input_fc, "id", """"comid_" + str(!FEATUREID_calc!) + "_block_" + str(!BLKIDFP00_calc!)""", "PYTHON3")
print("id calculated")


# Calculate the "name" field
arcpy.CalculateField_management(input_fc, "name", "!id!", "PYTHON3")
print("name calculated")

#delete the _calc fields
arcpy.DeleteField_management(input_fc, ["FEATUREID", "FEATUREID_calc", "BLKIDFP00_calc"])
print("fields deleted, now making centroids")

#rename the feature id field to the one that is an integer
arcpy.AlterField_management(input_fc, "FEATUREID_int", "FEATUREID")

#make a copy of the features on C drive
arcpy.management.CopyFeatures(input_fc, r"C:\Users\ewiggans\GeoConnexC\GeoConnex C Drive\GeoConnex C Drive.gdb\USA_2000_merge_final")

# Convert the polygon features to point centroids
arcpy.FeatureToPoint_management(input_fc, output_fc, "CENTROID")
print("You made centroids")


# Stop the timer
stop_time = time.time()

# Calculate the elapsed time
elapsed_time = stop_time - start_time
# Print a message for complete
print("completed successfully in " + str(round(elapsed_time, 2)) + " seconds.")